In [41]:
import pandas as pd
import numpy as np
from scipy.stats import chi2,norm

In [42]:
data = pd.read_csv("dataset\heart_2022_cleaned.csv")

In [43]:
data.shape

(354862, 40)

In [44]:
def do_chi2_test(data,column1,column2,alpha = 0.01):
    observed={}
    cols=data[column1].unique()
    for i in range(len(data[column2])):
        if data[column2].iloc[i] not in observed:
            observed[data[column2].iloc[i]]={}
            for j in cols:
                observed[data[column2].iloc[i]][j]=0
        observed[data[column2].iloc[i]][data[column1].iloc[i]]+=1

    observed=pd.DataFrame(observed).T
    observed=observed.sort_index()
    observed=observed[sorted(observed.columns)]
    print("Observed")
    print(observed)
    print()
    observed=observed.to_numpy()
    row_sums = observed.sum(axis=1)
    col_sums = observed.sum(axis=0)
    total = row_sums.sum()
    # expected = np.outer(row_sums, col_sums) / total

    expected=np.zeros(observed.shape)
    for i in range(observed.shape[0]):
        for j in range(observed.shape[1]):
            expected[i][j]=row_sums[i]*col_sums[j]/total
    exp=pd.DataFrame(expected)
    exp.columns=sorted(data[column1].unique())
    exp.index=sorted(data[column2].unique())
    print("Expected") 
    print(exp)
    chi2_stat = ((observed - expected)**2 / expected).sum()
   
    dof=(len(row_sums)-1)*(len(col_sums)-1)
    critical = chi2.ppf(1-alpha, dof)
    print("chi2_stat: ",chi2_stat)
    print("critical: ",critical)
    if chi2_stat>=critical:
        print("Dependent (reject H0)")
    else:
        print("Independent (fail to reject H0)")
    print()

### $\chi^2$ Test for independence between age category and Heart Health ###

In [45]:
do_chi2_test(data,'AgeCategory','General Health',0.01)

Observed
           Age 18 to 24  Age 25 to 29  Age 30 to 34  Age 35 to 39  \
Excellent          4777          3820          4271          4669   
Fair               2006          1678          1943          2188   
Good               6650          5409          6439          7185   
Poor                311           259           391           484   
Very good          8031          6658          7591          8448   

           Age 40 to 44  Age 45 to 49  Age 50 to 54  Age 55 to 59  \
Excellent          4709          4118          4552          4658   
Fair               2620          2878          3592          4394   
Good               7598          7221          8653          9058   
Poor                651           834          1232          1719   
Very good          8586          8029          9094          9930   

           Age 60 to 64  Age 65 to 69  Age 70 to 74  Age 75 to 79  \
Excellent          5342          5906          4292          2961   
Fair               5533

### Z-test for proportion ###

In [46]:
def do_proportion_test(data,column,threshold,alpha,po=0.5):
    p = len(data[column]>threshold)/len(data[column])
    z = (p-po)/np.sqrt(po*(1-po)/len(data[column]))
    print("z stat : ",z)
    critical_value = norm.ppf(1-alpha)
    print("critical_value : ",critical_value)

    if z<=critical_value:
        print("Reject H0")
    else:
        print("Fail to reject H0")

In [47]:
do_proportion_test(data,"BMI",25,0.05,0.4)  

z stat :  729.5841281168334
critical_value :  1.6448536269514722
Fail to reject H0


### One-Way ANOVA TEST ###

In [37]:
def perform_anova_test(data,column):
    groups = [data['BMI'][data['General Health'] == category] for category in data['General Health'].unique()]

    # Perform one-way ANOVA
    f_stat, p_value = stats.f_oneway(*groups)

    print("F-statistic:", f_stat)
    print("p-value:", p_value)


In [38]:
# perform_anova_test(data,'General Health',)

F-statistic: 5533.675227941374
p-value: 0.0
